In [1]:
from IPython.core.debugger import set_trace
from torchtext.datasets import WikiText2
import spacy
import re
import html
from torchtext import data
from spacy.symbols import ORTH
import torch
import torch.nn as nn
import torch.nn.functional as V
import torch.optim as optim

In [2]:
! ls data/

dogscats  dogscats.zip	wikitext-103  wikitext-2  wikitext-2-v1.zip


In [3]:
spacy_en  = spacy.load('en')

In [4]:
def tokenizer(x):
    return [tok.text for tok in spacy_en.tokenizer(x)]

In [5]:
TEXT = data.Field(lower=True, tokenize = tokenizer)


In [6]:
train, valid, test = WikiText2.splits(TEXT)

In [7]:
train

In [8]:
for ex in train.examples:
    print(ex.text[0:10])

[' ', '<eos>', ' ', '=', 'valkyria', 'chronicles', 'iii', '=', '<eos>', ' ']


In [9]:
TEXT.build_vocab(train, vectors = "fasttext.en.300d")

In [10]:
TEXT

In [11]:
train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=32,
    bptt_len=30, # this is where we specify the sequence length
    device = "cuda",
    repeat=False)

In [12]:
for batch in train_iter:
    print(batch.text.shape)
    #print("text")
    #print(batch.text.data)
    #print("target")
    #print(batch.target.data)
    break

torch.Size([30, 32])


In [13]:
batch.text.data[:,0]

tensor([   12,    13,    12,    15,  3875,  3895,   889,    15,    13,    12,
           13,    12, 20060,    93,  3875,    98,    52,     6,     8,     7,
         3895,    27,   789,    52, 28868,     3,  6216,     4,  3875,     5],
       device='cuda:0')

In [14]:
batch.target.data[:,0]

tensor([   13,    12,    15,  3875,  3895,   889,    15,    13,    12,    13,
           12, 20060,    93,  3875,    98,    52,     6,     8,     7,  3895,
           27,   789,    52, 28868,     3,  6216,     4,  3875,     5,     2],
       device='cuda:0')

In [97]:
class LanguageModel(nn.Module):
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz,
                 dropout=0.5):
        super(LanguageModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid,ntoken)

        self.init_weights()
        self.hidden = self.init_hidden(bsz)
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
 
    def forward(self, input):
        emb = self.drop(self.encoder(input))
        output, self.hidden = self.rnn(emb, self.hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        decoded_viewed = decoded.view(output.size(0), output.size(1), decoded.size(1))
        return decoded_viewed
 
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return (torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_().cuda()),
                torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_()).cuda())
  
    def reset_history(self):
        self.hidden = tuple(torch.tensor(v.data) for v in self.hidden)

In [102]:
weight_matrix = TEXT.vocab.vectors
model = LanguageModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 1, 32)
model.encoder.weight.data.copy_(weight_matrix)
model.cuda()

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


LanguageModel(
  (drop): Dropout(p=0.5)
  (encoder): Embedding(28870, 300)
  (rnn): LSTM(300, 200, dropout=0.5)
  (decoder): Linear(in_features=200, out_features=28870, bias=True)
  (softmax): Softmax()
)

In [103]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3, betas=(0.9,0.999))
n_tokens = weight_matrix.size(0)

In [104]:
from tqdm import tqdm
def train_epoch(epoch):
    epoch_loss = 0
    for batch in tqdm(train_iter):
        model.reset_history()
    
        optimizer.zero_grad()
        
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        loss.backward()
        
        optimizer.step()
        
        batch_loss = loss.item() * prediction.size(0) * prediction.size(1)
        
        batch_loss /= len(train.examples[0].text)
        
        epoch_loss += batch_loss
        

    val_loss = 0
    for batch in valid_iter:
        model.reset_history()
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        batch_loss = loss.item() * text.size(0)
        batch_loss /= len(valid.examples[0].text) 
        val_loss += batch_loss
        
        
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))
        

In [105]:
for i in range(2):
    train_epoch(i)

  0%|          | 0/2330 [00:00<?, ?it/s]

Epoch: 0, Training Loss: 10.2135, Validation Loss: 0.3191


100%|██████████| 2330/2330 [01:10<00:00, 32.86it/s]


Epoch: 1, Training Loss: 10.2121, Validation Loss: 0.3191


In [120]:
import math

def perplexity(text, prob_of):
    total_prob_log = 1
    index = 0
    for word in text:
        total_prob_log += math.log(prob_of(word, index))
        index += 1
    total_prob = math.exp(total_prob_log)
    print(total_prob)
    return math.pow(total_prob, -1/len(text))

def single_batch(words, length):
    return [[word] for word in words[:length]]

def prob_of_model(model, text, word, index):
    result_all = model(torch.tensor(single_batch(text, index + 1)).cuda())
    result_values = result_all[result_all.shape[0]-1,0,:];
    result_softmax = nn.Softmax(0)(result_values)
    result = result_softmax[word].item()
    print(result)
    return result
    
def model_perplexity(text, model):
    model.reset_history()
    model.hidden = model.init_hidden(1)
    prob_of = lambda word, index: prob_of_model(model, text, word, index)
    return perplexity(text, prob_of)

In [130]:
model_perplexity([13, 12, 15, 3875, 3895, 889], model)

3.463833490968682e-05
3.463659595581703e-05
3.463595930952579e-05
3.463595930952579e-05
3.463597749941982e-05
3.4635962947504595e-05
4.693489483747126e-27


24439.03341276804

In [ ]:
perplexity("test test test", lambda x, index: 0.01)